In [1]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from zipfile import ZipFile
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

In [2]:
#JUPYTER VERSION
#comb_path = "../Data/Combined_Images"

#GOOGLE COLAB VERSION
from google.colab import drive
drive.mount('/content/drive')

# Unzipping the dataset file facial-age.zip

combined_images_path = "/content/drive/MyDrive/Data/Combined_Images.zip"

with ZipFile(combined_images_path, 'r') as myzip:
    myzip.extractall("../content/Combined_Images")
    print('Done unzipping Combined_Images.zip')

Mounted at /content/drive
Done unzipping Combined_Images.zip


In [3]:
comb_path = '../content/Combined_Images' 
batch_size = 64

train_ds = tf.keras.utils.image_dataset_from_directory(
  comb_path,
  validation_split=0.2,
  subset="training", #If should be return the training set (80%) or the validation set (20%)
  seed=41, #Seed should guarantee that train_ds and val_ds doesn't have common images
  shuffle=True,
  image_size=(200, 200),
  batch_size=batch_size,
  color_mode='grayscale')

val_ds = tf.keras.utils.image_dataset_from_directory(
  comb_path,
  validation_split=0.2,
  subset="validation",
  seed=41, 
  shuffle=True,
  image_size=(200, 200),
  batch_size=batch_size,
  color_mode='grayscale')

test_dataset = val_ds.take(53)
val_ds = val_ds.skip(53)

print('Batches for training -->', train_ds.cardinality())
print('Batches for validating -->', val_ds.cardinality())
print('Batches for testing -->', test_dataset.cardinality())

Found 33884 files belonging to 8 classes.
Using 27108 files for training.
Found 33884 files belonging to 8 classes.
Using 6776 files for validation.
Batches for training --> tf.Tensor(424, shape=(), dtype=int64)
Batches for validating --> tf.Tensor(53, shape=(), dtype=int64)
Batches for testing --> tf.Tensor(53, shape=(), dtype=int64)


## ALEXNET

In [4]:
model4 = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(200, 200, 1)),
    tf.keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(200,200,1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(8, activation='softmax')
])

#model.compile(optimizer=tf.keras.optimizers.Adam(),
              #loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              #metrics=tf.keras.metrics.Accuracy())

#Additionaly, if you do not one-hot encode your data, set sparse_categorical_crossentropy as loss and sparse_categorical_accuracy as metric.
model4.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])

# Defining the early stop to monitor the validation loss to avoid overfitting.
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')

epochs=25
history = model4.fit(
  train_ds,
  validation_data=val_ds,
  callbacks=[early_stop],
  epochs=epochs,
  shuffle=True
)

Epoch 1/25
424/424 [==============================] - 56s 99ms/step - loss: 2.8574 - sparse_categorical_accuracy: 0.2176 - val_loss: 1.9688 - val_sparse_categorical_accuracy: 0.2408
Epoch 2/25
424/424 [==============================] - 39s 92ms/step - loss: 1.9504 - sparse_categorical_accuracy: 0.2458 - val_loss: 1.8950 - val_sparse_categorical_accuracy: 0.2503
Epoch 3/25
424/424 [==============================] - 39s 91ms/step - loss: 1.8848 - sparse_categorical_accuracy: 0.2668 - val_loss: 2.0308 - val_sparse_categorical_accuracy: 0.1823
Epoch 4/25
424/424 [==============================] - 39s 91ms/step - loss: 1.8566 - sparse_categorical_accuracy: 0.2807 - val_loss: 1.8079 - val_sparse_categorical_accuracy: 0.3050
Epoch 5/25
424/424 [==============================] - 39s 90ms/step - loss: 1.8227 - sparse_categorical_accuracy: 0.2914 - val_loss: 1.8876 - val_sparse_categorical_accuracy: 0.2686
Epoch 6/25
424/424 [==============================] - 38s 89ms/step - loss: 1.7790 - spars

In [5]:
print("MODEL 4 --- ALEXNET")
model4.evaluate(test_dataset)

MODEL 4 --- ALEXNET
53/53 [==============================] - 4s 56ms/step - loss: 1.7779 - sparse_categorical_accuracy: 0.4092


[1.7778868675231934, 0.4091981053352356]